In [4]:
import cv2
import dlib
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim
from torchvision import transforms
from PIL import Image, ImageOps

import matplotlib.pyplot as plt 
import numpy as np
from PIL import Image

import os
import shutil
import time

In [2]:
# Load pre-trained face detector
modelFile = "../Pretrained Detectors/res10_300x300_ssd_iter_140000.caffemodel"
configFile = "../Pretrained Detectors/deploy.prototxt"
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)

# Load pre-trained landmark predictor
predictor = dlib.shape_predictor("../Pretrained Detectors/shape_predictor_68_face_landmarks.dat")


In [5]:
# Creates and empties a folder called "Frames"

path = "../Frames"
shutil.rmtree(path, ignore_errors = True)
if not os.path.exists(path):
    os.makedirs("../Frames")

In [6]:
def detectFace(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)     
    blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))

    net.setInput(blob)
    detections = net.forward()

    (x1, y1, x2, y2) = 0, 0, 0, 0
    maxConfidence = 0

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        if confidence > 0.5 and confidence > maxConfidence:
            (h, w) = img.shape[:2]
            x1 = int(detections[0, 0, i, 3] * w)
            y1 = int(detections[0, 0, i, 4] * h)
            x2 = int(detections[0, 0, i, 5] * w)
            y2 = int(detections[0, 0, i, 6] * h)

            maxConfidence = confidence

    return (dlib.rectangle(x1, y1, x2, y2), maxConfidence)


In [8]:
counter = 0
cap = cv2.VideoCapture("../Datasets/DROZY/videos_i8/5-1.mp4")    
#cap = cv2.VideoCapture("../Datasets/X/01/0.mov")   
#cap = cv2.VideoCapture(0)   

while cap.isOpened():
    ret, frame = cap.read()     # return status and image

    if not ret:
        print("Can't retreive frame")
        break
    
    t = time.time()
    face, confidence = detectFace(frame)
    print(time.time() - t)
    t = time.time()

    if confidence > 0:
        landmarks = predictor(frame, face)

        x1 = landmarks.part(17).x
        x2 = landmarks.part(21).x
        d = abs(x2-x1)
        k = d * 0.15

        x1 = x1 - int(k/2)
        x2 = x2 + int(k/2)
        y1 = landmarks.part(19).y - int(k/2)
        y2 = y1 + int(d+k)
        

        # Save frame as image
        eye = frame[y1:y2, x1:x2]
        cv2.imwrite("./Frames/" + str(counter) + ".jpg", eye)       # Save frame images inside folder
        counter += 1

        # Visuals
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)

        cv2.rectangle(frame, (face.left(), face.top()), (face.right(), face.bottom()), (255, 0, 0), 2)
        text = "{:.2f}%".format(confidence * 100)
        cv2.putText(frame, text, (face.left(), face.top()), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 1)

    # Display Output
    cv2.imshow("img", frame)

    # Exit windows using "q" key
    if cv2.waitKey(1) == ord("q"):
        break


cap.release()
cv2.destroyAllWindows()

0.03000664710998535
0.02400517463684082
0.024005413055419922
0.03000664710998535
0.031007051467895508
0.028006553649902344
0.027005910873413086
0.028006553649902344
0.02800583839416504
0.029006242752075195
0.02700662612915039
0.028006315231323242
0.027007102966308594
0.027006864547729492
0.0290067195892334
0.0290067195892334
0.028006553649902344
0.028006792068481445
0.027006149291992188
0.02800726890563965
0.03300762176513672
0.027006149291992188
0.027006149291992188
0.02700638771057129
0.028006553649902344
0.027006149291992188
0.027005672454833984
0.02700662612915039
0.028004884719848633
0.0290069580078125
0.028006553649902344
0.026006221771240234
0.027006149291992188
0.024005651473999023
0.023004531860351562
0.027005910873413086
0.02300572395324707
0.025005578994750977
0.02400493621826172
0.023004531860351562
0.028006553649902344
0.02700662612915039
0.02700638771057129
0.029006242752075195
0.027005672454833984
0.027006149291992188
0.028005599975585938
0.026006221771240234
0.026005506